#Taller Final
###Nombre estudiante: Daniela Mancilla

En este taller final, se espera que haga uso de transacciones para mantener un sistema de arriendo de herramientas. Hay tres tablas en la base de datos, de las cuales se les muestran sus respectivos `CREATE TABLE` para que conozca sus nombres y columnas.

### Tabla cliente

Tiene una columna id que es la llave primaria y un nombre

```sql
CREATE TABLE cliente (
    id integer primary key,
    nombre varchar(200)
)
```

### Tabla herramienta

Tiene una columna id que es la llave primaria, un nombre de la herramienta, un stock que no puede ser menor que 0 y un precio por día.

```sql
CREATE TABLE herramienta (
    id integer primary key,
    nombre varchar(200),
    stock integer check (stock >= 0),
    precio_diario float
)
```

### Tabla arriendo

Tiene el id de la herramienta arrendada, el id de el cliente que arrienda, el timestamp (tiempo actual) en que se realiza el arriendo y la cantidad de días que durará el arriendo. La llave primaria está conformada por las tres primeras columnas

```sql
CREATE TABLE arriendo (
    cliente integer REFERENCES cliente(id),
    herramienta integer REFERENCES herramienta(id),
    fecha TIMESTAMP,
    dias integer,
    PRIMARY KEY (cliente, herramienta, fecha)
)
```

## Lo que Ud. debe realizar:

Primero debe mantener un objeto de conexión a la base de datos del curso y generar un cursor, para hacer consultas de tipo `INSERT` y `UPDATE`

In [1]:
import psycopg2

db_dict = {
    'host': 'lin-29996-12677-pgsql-primary.servers.linodedb.net',
    'database': 'viajes',
    'user': 'diplo',
    'password': 'diplo_python',
    'port': '5432'
}
conn = psycopg2.connect(**db_dict)
cursor = conn.cursor()

Luego puede ver los datos disponibles:

In [2]:
cursor.execute("SELECT * FROM herramienta")
for r in cursor.fetchall():
  print(r)

(1, 'martillo', 0, 500.0)
(2, 'sierra electrica', 1, 7000.0)
(4, 'lijadora', 0, 3000.0)
(5, 'soldadora', 3, 12000.0)
(6, 'nivel laser', 4, 4000.0)
(3, 'taladro', 8, 2500.0)


In [3]:
cursor.execute("SELECT * FROM cliente")
for r in cursor.fetchall():
  print(r)

(1, 'Pedro')
(2, 'Juana')
(3, 'Isabel')
(4, 'Ernesto')
(5, 'Mario')
(6, 'Ana')
(7, 'Emilio')
(8, 'Sara')


En la siguiente celda, pida un id de herramienta (usando `input`) y luego un
id de cliente. Debe verificar que existe una herramienta en la tabla Herramienta con el id ingresado. Debe hacer la misma verificación con el id de cliente en la tabla cliente.

En caso de no encontrar al cliente o a la herramienta, muestre un mensaje de error.

Si se encuentra tanto el cliente como la herramienta, debe pedir ingresar la cantidad de días de duración del arriendo. Verifique que el número ingresado es mayor o igual a 1 y, de ser así, imprima el precio a pagar. Si todo está
correcto inserte una fila en la tabla Arriendo, con el id de la herramienta, el id del cliente, la fecha y hora actual, y la cantidad de días. No olvide hacer commit.

Verifique que su código use sentencias precompiladas, para que no sea posible realizar inyecciones SQL.

In [9]:
from datetime import datetime

In [30]:
def arrendar():
  id_herramienta = input("Ingrese ID de herramienta: ")
  cursor.execute("SELECT id FROM herramienta WHERE id=%s", (id_herramienta,))
  if len(cursor.fetchall())==0:
    print("No existe la herramienta "+id_herramienta)
    conn.rollback()  #hacer rollback en caso de error
    return
  #validar id cliente
  id_cliente = input("Ingrese ID cliente: ")
  cursor.execute("SELECT id FROM cliente WHERE id=%s", (id_cliente,))
  if len(cursor.fetchall())==0:
    print("No existe el cliente "+id_cliente)
    conn.rollback()  #hacer rollback en caso de error
    return
  #pedir días de arriendo y verificar que es mayor o igual que 1
  dias = input("Ingrese los días de arriendo: ")
  if int(dias)<1:
    print("El número de días ingresado no es válido")
    conn.rollback()  #hacer rollback en caso de error
    return
  #imprima precio a pagar
  cursor.execute("SELECT precio_diario FROM herramienta WHERE id=%s", (id_herramienta,))
  precio_diario=cursor.fetchall()[0][0]
  precio_pagar=int(dias)*precio_diario
  print("El precio a pagar es: " + str(precio_pagar))
  #inserte en la tabla arriedo
  fecha=datetime.now()
  cursor.execute("INSERT INTO arriendo VALUES (%s,%s,%s,%s)",(id_cliente,id_herramienta, fecha, dias,))
  #haga commit (conn.commit())
  conn.commit()

In [31]:
#ejecutamos la función
arrendar()

Ingrese ID de herramienta: 3
Ingrese ID cliente: 1
Ingrese los días de arriendo: 100
El precio a pagar es: 250000.0


In [11]:
conn.rollback()

Usted habrá notado que no hemos verificado que haya stock de la herramienta solicitada. Agregue una validación que revise si el stock es mayor o igual a uno antes de pedir el id del cliente.

In [32]:
def arrendar2():
  id_herramienta = input("Ingrese ID de herramienta :")
  cursor.execute("SELECT id FROM herramienta WHERE id=%s", (id_herramienta,))
  if len(cursor.fetchall())==0:
    print("No existe la herramienta "+id_herramienta)
    conn.rollback()  #hacer rollback en caso de error
    return
  #verificar stock
  cursor.execute("SELECT stock FROM herramienta WHERE id=%s", (id_herramienta,))
  stock=cursor.fetchall()[0][0]
  if stock<1:
    print("No hay suficiente stock")
    conn.rollback()  #hacer rollback en caso de error
    return
  #validar id cliente
  id_cliente = input("Ingrese ID cliente: ")
  cursor.execute("SELECT id FROM cliente WHERE id=%s", (id_cliente,))
  if len(cursor.fetchall())==0:
    print("No existe el cliente "+id_cliente)
    conn.rollback()  #hacer rollback en caso de error
    return
  #pedir días de arriendo y verificar que es mayor o igual que 1
  dias = input("Ingrese los días de arriendo: ")
  if int(dias)<1:
    print("El número de días ingresado no es válido")
    conn.rollback()  #hacer rollback en caso de error
    return
  #imprima precio a pagar
  cursor.execute("SELECT precio_diario FROM herramienta WHERE id=%s", (id_herramienta,))
  precio_diario=cursor.fetchall()[0][0]
  precio_pagar=int(dias)*precio_diario
  print("El precio a pagar es: " + str(precio_pagar))
  #inserte en la tabla arriedo
  fecha=datetime.now()
  cursor.execute("INSERT INTO arriendo VALUES (%s,%s,%s,%s)",(id_cliente,id_herramienta, fecha, dias,))
  #haga commit (conn.commit())
  conn.commit()

In [33]:
#probamos la nueva función
arrendar2()

Ingrese ID de herramienta :1
No hay suficiente stock


In [ ]:
conn.rollback()

Finalmente, usted notará que también es necesario reducir el stock en la tabla herramienta, cada vez que se inserta una fila en la tabla Arriendo.

A continuación, explique los problemas que pueden surgir al hacer el insert en Arriendo y el update en Herramienta sin usar transacciones:

Si no usamos transacciones, y dos usuarios quisieran arrendar al mismo tiempo, la misma herramienta que tenga stock 1, a quién se la damos? Por esta razón es necesario actualizar el stock al mismo tiempo que el arriendo, de lo contrario, al ir a buscar la herramienta a la bodega, podríamos encontrarnos con que ya fue arrendada por otro cliente.

Haga una implementacion final que inserte el arriendo y actualice el stock en la misma transacción:

In [34]:
def arrendar3():
  id_herramienta = input("Ingrese ID de herramienta :")
  cursor.execute("SELECT id FROM herramienta WHERE id=%s", (id_herramienta,))
  if len(cursor.fetchall())==0:
    print("No existe la herramienta "+id_herramienta)
    conn.rollback()  #hacer rollback en caso de error
    return
  #verificar stock
  cursor.execute("SELECT stock FROM herramienta WHERE id=%s", (id_herramienta,))
  stock=cursor.fetchall()[0][0]
  if stock<1:
    print("No hay suficiente stock")
    conn.rollback()  #hacer rollback en caso de error
    return
  #validar id cliente
  id_cliente = input("Ingrese ID cliente: ")
  cursor.execute("SELECT id FROM cliente WHERE id=%s", (id_cliente,))
  if len(cursor.fetchall())==0:
    print("No existe el cliente "+id_cliente)
    conn.rollback()  #hacer rollback en caso de error
    return
  #pedir días de arriendo y verificar que es mayor o igual que 1
  dias = input("Ingrese los días de arriendo: ")
  if int(dias)<1:
    print("El número de días ingresado no es válido")
    conn.rollback()  #hacer rollback en caso de error
    return
  #imprima precio a pagar
  cursor.execute("SELECT precio_diario FROM herramienta WHERE id=%s", (id_herramienta,))
  precio_diario=cursor.fetchall()[0][0]
  precio_pagar=int(dias)*precio_diario
  print("El precio a pagar es: " + str(precio_pagar))
  #inserte en la tabla arriedo
  fecha=datetime.now()
  #d = fecha.strftime()
  cursor.execute("INSERT INTO arriendo VALUES (%s,%s,%s,%s)",(id_cliente,id_herramienta, fecha, dias,))
  #actualice el stock en la tabla herramienta
  cursor.execute("SELECT stock FROM herramienta WHERE id=%s", (id_herramienta,))
  new_stock=cursor.fetchall()[0][0]-1
  cursor.execute("UPDATE herramienta SET stock=%s WHERE id=%s",(new_stock,id_herramienta,))
  cursor.execute("SELECT * FROM herramienta WHERE id=%s",(id_herramienta,))
  #haga commit (conn.commit())
  conn.commit()

In [35]:
#probamos la función final
arrendar3()

Ingrese ID de herramienta :5
Ingrese ID cliente: 3
Ingrese los días de arriendo: 1000
El precio a pagar es: 12000000.0


In [36]:
#probamos la función final
arrendar3()

Ingrese ID de herramienta :1
No hay suficiente stock


In [37]:
#probamos la función final
arrendar3()

Ingrese ID de herramienta :3
Ingrese ID cliente: 10
No existe el cliente 10


In [40]:
#probamos la función final
arrendar3()

Ingrese ID de herramienta :3
Ingrese ID cliente: 8
Ingrese los días de arriendo: 20
El precio a pagar es: 50000.0


In [26]:
conn.rollback()